In [6]:
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np


In [7]:
# Load the BCCD dataset without `as_supervised=True`
dataset, info = tfds.load('bccd', with_info=True)

# Access the training and testing splits
train_dataset = dataset['train']
test_dataset = dataset['test']

# Inspect the structure of a single example from the dataset
example = next(iter(train_dataset))
print(example)


{'image': <tf.Tensor: shape=(480, 640, 3), dtype=uint8, numpy=
array([[[183, 184, 179],
        [184, 185, 179],
        [185, 186, 180],
        ...,
        [176, 143, 136],
        [176, 143, 138],
        [176, 143, 138]],

       [[183, 184, 179],
        [184, 185, 179],
        [185, 186, 178],
        ...,
        [177, 144, 137],
        [177, 144, 139],
        [177, 144, 139]],

       [[182, 184, 179],
        [183, 186, 179],
        [184, 187, 178],
        ...,
        [177, 144, 137],
        [177, 144, 139],
        [177, 144, 139]],

       ...,

       [[195, 199, 184],
        [195, 199, 184],
        [195, 199, 185],
        ...,
        [198, 203, 196],
        [198, 204, 194],
        [197, 203, 193]],

       [[195, 199, 184],
        [195, 199, 184],
        [195, 199, 185],
        ...,
        [199, 205, 195],
        [198, 204, 194],
        [198, 204, 192]],

       [[195, 199, 184],
        [195, 199, 184],
        [195, 199, 185],
        ...,
        [19

In [8]:
# a. Show the number of training and testing images
train_size = info.splits['train'].num_examples
test_size = info.splits['test'].num_examples
print(f'Training images: {train_size}')
print(f'Testing images: {test_size}')


Training images: 205
Testing images: 72


In [10]:
# Inspecting a single example from the dataset to find the correct keys
example = next(iter(train_dataset))
print(example)


{'image': <tf.Tensor: shape=(480, 640, 3), dtype=uint8, numpy=
array([[[183, 184, 179],
        [184, 185, 179],
        [185, 186, 180],
        ...,
        [176, 143, 136],
        [176, 143, 138],
        [176, 143, 138]],

       [[183, 184, 179],
        [184, 185, 179],
        [185, 186, 178],
        ...,
        [177, 144, 137],
        [177, 144, 139],
        [177, 144, 139]],

       [[182, 184, 179],
        [183, 186, 179],
        [184, 187, 178],
        ...,
        [177, 144, 137],
        [177, 144, 139],
        [177, 144, 139]],

       ...,

       [[195, 199, 184],
        [195, 199, 184],
        [195, 199, 185],
        ...,
        [198, 203, 196],
        [198, 204, 194],
        [197, 203, 193]],

       [[195, 199, 184],
        [195, 199, 184],
        [195, 199, 185],
        ...,
        [199, 205, 195],
        [198, 204, 194],
        [198, 204, 192]],

       [[195, 199, 184],
        [195, 199, 184],
        [195, 199, 185],
        ...,
        [19

In [9]:
# Adjust the plotting function based on the actual keys in the dataset
def plot_images(dataset, num_images=5):
    plt.figure(figsize=(10, 10))
    for i, example in enumerate(dataset.take(num_images)):
        image = example['image']  # Assuming 'image' is the key
        label = example['label']  # Assuming 'label' is the key
        ax = plt.subplot(1, num_images, i + 1)
        plt.imshow(image)
        plt.title(f'Class: {label.numpy()}')  # Use label as an integer or category name
        plt.axis('off')
    plt.show()

plot_images(train_dataset)


KeyError: 'label'

<Figure size 1000x1000 with 0 Axes>

In [11]:
# c. Plot some sample images after identifying the correct label key
def plot_images(dataset, num_images=5):
    plt.figure(figsize=(10, 10))
    for i, example in enumerate(dataset.take(num_images)):
        image = example['image']  # The image key
        labels = example['label']  # The label key containing multiple labels for objects
        ax = plt.subplot(1, num_images, i + 1)
        plt.imshow(image)
        plt.title(f'Class: {labels.numpy()}')  # Displaying the labels for objects in the image
        plt.axis('off')
    plt.show()

plot_images(train_dataset)


KeyError: 'label'

<Figure size 1000x1000 with 0 Axes>

In [ ]:
# c. Image Augmentation (contrast, flipping, and rotation)
def augment_image(image, label):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_contrast(image, lower=0.7, upper=1.3)
    image = tf.image.random_rotation(image, 90)
    return image, label

train_dataset = train_dataset.map(lambda x: augment_image(x['image'], x['label']), num_parallel_calls=tf.data.experimental.AUTOTUNE)
test_dataset = test_dataset.map(lambda x: augment_image(x['image'], x['label']), num_parallel_calls=tf.data.experimental.AUTOTUNE)


In [ ]:
# d. Show the number of training and testing images after augmentation
train_size_after_aug = sum(1 for _ in train_dataset)
test_size_after_aug = sum(1 for _ in test_dataset)
print(f'Training images after augmentation: {train_size_after_aug}')
print(f'Testing images after augmentation: {test_size_after_aug}')


In [ ]:
# e. Normalize the training data
def normalize_image(image, label):
    image = tf.cast(image, tf.float32)
    image = image / 255.0  # Normalize to [0, 1] range
    return image, label

train_dataset = train_dataset.map(normalize_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)
test_dataset = test_dataset.map(normalize_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)


In [ ]:
# f. Build a convolutional neural network (CNN)
def build_cnn_model():
    model = tf.keras.Sequential([
        tf.keras.layers.InputLayer(input_shape=(256, 256, 3)),
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(3, activation='softmax')  # 3 classes: WBC, RBC, Platelets
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model


In [ ]:
# g. Train the model without augmentation
cnn_model = build_cnn_model()
cnn_model.fit(train_dataset.batch(32), epochs=10, validation_data=test_dataset.batch(32))


In [ ]:
# h. Show training and testing accuracy
train_accuracy = cnn_model.history.history['accuracy']
test_accuracy = cnn_model.history.history['val_accuracy']

plt.plot(train_accuracy, label='Training Accuracy')
plt.plot(test_accuracy, label='Testing Accuracy')
plt.title('Training and Testing Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()


In [ ]:
# i. Train the model again (with augmentation and normalization included)
cnn_model = build_cnn_model()
cnn_model.fit(train_dataset.batch(32), epochs=10, validation_data=test_dataset.batch(32))


In [ ]:
# j. Show the training and testing accuracy after retraining
train_accuracy_aug = cnn_model.history.history['accuracy']
test_accuracy_aug = cnn_model.history.history['val_accuracy']

plt.plot(train_accuracy_aug, label='Training Accuracy (Augmented)')
plt.plot(test_accuracy_aug, label='Testing Accuracy (Augmented)')
plt.title('Training and Testing Accuracy (With Augmentation)')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()


In [ ]:
# k. Compare the training and testing accuracy before and after augmentation
plt.plot(train_accuracy, label='Training Accuracy (Before Augmentation)')
plt.plot(test_accuracy, label='Testing Accuracy (Before Augmentation)')
plt.plot(train_accuracy_aug, label='Training Accuracy (After Augmentation)')
plt.plot(test_accuracy_aug, label='Testing Accuracy (After Augmentation)')
plt.title('Training and Testing Accuracy Comparison')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()
